In [79]:
import pandas as pd
import numpy as np


In [75]:
df = pd.read_excel("keyword_rows.xlsx")


In [76]:
df.columns

Index(['Energy', 'Materials', 'Industrials', 'Consumer Discretionary',
       'Consumer Staples', 'Health Care', 'Financials',
       'Information Technology', 'Communication Services', 'Utilities',
       'Real Estate'],
      dtype='object')

In [78]:

df.head(5)

df.Energy

0                                    Oil & Gas Drilling
1     Drilling contractors or owners of drilling rig...
2                        Oil & Gas Equipment & Services
3     Manufacturers of equipment, including drilling...
4                                  Integrated Oil & Gas
5     Integrated oil companies engaged in the explor...
6                    Oil & Gas Exploration & Production
7     Companies engaged in the exploration and produ...
8                        Oil & Gas Refining & Marketing
9     Companies engaged in the refining and marketin...
10                   Oil & Gas Storage & Transportation
11    Companies engaged in the storage and/or transp...
12                              Coal & Consumable Fuels
13    Companies primarily involved in the production...
14                                                  NaN
15                                                  NaN
16                                                  NaN
17                                              

In [74]:
import re
def pre_process(text):
    
    # lowercase
    text=text.lower()
    
    #remove tags
    text=re.sub("</?.*?>"," <> ",text)
    
    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)
    
    return text

df['text'] =df['Energy']
df['text'] = df['text'].apply(lambda x:pre_process(x))




KeyError: 'Energy'

In [52]:
from sklearn.feature_extraction.text import CountVectorizer
import re
from nltk.corpus import stopwords



#load a set of stop words
stopwords=set(stopwords.words("english"))
#get the text column 
docs=df['text'].tolist()

#create a vocabulary of words, 
#ignore words that appear in 85% of documents, 
#eliminate stop words
cv=CountVectorizer(max_df=0.85,stop_words=stopwords)
word_count_vector=cv.fit_transform(docs)

In [53]:
word_count_vector.shape

(330, 836)

In [54]:
cv=CountVectorizer(max_df=0.85,stop_words=stopwords,max_features=10000)
word_count_vector=cv.fit_transform(docs)
word_count_vector.shape

(330, 836)

In [55]:
list(cv.vocabulary_.keys())[:10]

['oil',
 'gas',
 'drilling',
 'contractors',
 'owners',
 'rigs',
 'contract',
 'services',
 'wells',
 'equipment']

In [56]:
list(cv.get_feature_names())[:20]

['access',
 'accessories',
 'acquisition',
 'acquisitions',
 'across',
 'activities',
 'activity',
 'added',
 'addition',
 'additives',
 'adhesives',
 'advanced',
 'advertisements',
 'advertising',
 'advisory',
 'aerospace',
 'african',
 'agencies',
 'agency',
 'agents']

In [57]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

In [58]:
tfidf_transformer.idf_

array([6.10897119, 4.85620823, 4.72267683, 6.10897119, 5.70350609,
       4.72267683, 5.41582401, 6.10897119, 6.10897119, 6.10897119,
       6.10897119, 6.10897119, 6.10897119, 5.01035891, 6.10897119,
       5.70350609, 6.10897119, 5.41582401, 6.10897119, 5.70350609,
       6.10897119, 4.72267683, 6.10897119, 5.41582401, 5.70350609,
       6.10897119, 6.10897119, 5.70350609, 5.70350609, 4.09406817,
       6.10897119, 5.41582401, 6.10897119, 6.10897119, 5.70350609,
       6.10897119, 6.10897119, 5.01035891, 5.41582401, 5.19268046,
       5.70350609, 6.10897119, 6.10897119, 6.10897119, 6.10897119,
       5.41582401, 5.70350609, 6.10897119, 6.10897119, 6.10897119,
       5.41582401, 5.70350609, 6.10897119, 6.10897119, 5.70350609,
       5.70350609, 6.10897119, 6.10897119, 6.10897119, 4.72267683,
       4.31721173, 6.10897119, 6.10897119, 5.01035891, 6.10897119,
       5.70350609, 6.10897119, 6.10897119, 6.10897119, 6.10897119,
       6.10897119, 5.70350609, 5.70350609, 6.10897119, 5.41582

In [61]:
df['text'] =  df['Sector_Description']

df['text'] =df['text'].apply(lambda x:pre_process(x))

docs_test=df['text'].tolist()

In [62]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []

    for idx, score in sorted_items:
        fname = feature_names[idx]
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

In [63]:


# you only needs to do this once
feature_names=cv.get_feature_names()

# get the document that we want to extract keywords from
doc=docs_test[1]

#generate tf-idf for the given document
tf_idf_vector=tfidf_transformer.transform(cv.transform([doc]))

#sort the tf-idf vectors by descending order of scores
sorted_items=sort_coo(tf_idf_vector.tocoo())

#extract only the top n; n here is 10
keywords=extract_topn_from_vector(feature_names,sorted_items,10)

# now print the results
print("\n=====Title=====")
print(docs_test[0])
print("\n===Keywords===")
for k in keywords:
    print(k,keywords[k])


=====Title=====
oil gas drilling

===Keywords===
drilling 0.798
wells 0.28
rigs 0.28
contractors 0.28
contract 0.28
owners 0.18
services 0.128


In [15]:
docs_test

['oil gas drilling',
 'drilling contractors or owners of drilling rigs that contract their services for drilling wells',
 'oil gas equipment services',
 'manufacturers of equipment including drilling rigs and equipment and providers of supplies and services to companies involved in the drilling evaluation and completion of oil and gas wells ',
 'integrated oil gas',
 'integrated oil companies engaged in the exploration production of oil and gas as well as at least one other significant activity in either refining marketing and transportation or chemicals ',
 'oil gas exploration production',
 'companies engaged in the exploration and production of oil and gas not classified elsewhere ',
 'oil gas refining marketing',
 'companies engaged in the refining and marketing of oil gas and or refined products not classified in the integrated oil gas or independent power producers energy traders sub industries ',
 'oil gas storage transportation',
 'companies engaged in the storage and or transp

In [16]:
# put the common code into several methods
def get_keywords(idx):

    #generate tf-idf for the given document
    tf_idf_vector=tfidf_transformer.transform(cv.transform([docs_test[idx]]))

    #sort the tf-idf vectors by descending order of scores
    sorted_items=sort_coo(tf_idf_vector.tocoo())

    #extract only the top n; n here is 10
    keywords=extract_topn_from_vector(feature_names,sorted_items,10)
    
    return keywords

def print_results(idx,keywords):
    # now print the results
    print("\n=====Title=====")
    print(docs_test[idx])
    print("\n===Keywords===")
    for k in keywords:
        print(k,keywords[k])

In [17]:
idx=329
keywords=get_keywords(idx)
print_results(idx,keywords)


=====Title=====
real estate service providers such as real estate agents brokers real estate appraisers 

===Keywords===
real 0.597
estate 0.597
appraisers 0.27
brokers 0.252
agents 0.252
service 0.24
providers 0.171


In [18]:
idx=16
keywords=get_keywords(idx)
print(docs_test[16])
print_results(idx,keywords)


diversified chemicals

=====Title=====
diversified chemicals

===Keywords===
chemicals 0.771
diversified 0.636


In [19]:
dict_new = {}
idx =1
while(idx<320):
    keywords=get_keywords(idx)
    print(docs_test[idx])
    print(keywords)
    dict_new.update(keywords)
    idx=idx+1


    

drilling contractors or owners of drilling rigs that contract their services for drilling wells
{'drilling': 0.798, 'wells': 0.28, 'rigs': 0.28, 'contractors': 0.28, 'contract': 0.28, 'owners': 0.18, 'services': 0.128}
oil gas equipment services
{'oil': 0.589, 'gas': 0.551, 'equipment': 0.468, 'services': 0.361}
manufacturers of equipment including drilling rigs and equipment and providers of supplies and services to companies involved in the drilling evaluation and completion of oil and gas wells 
{'drilling': 0.525, 'equipment': 0.327, 'evaluation': 0.296, 'completion': 0.296, 'wells': 0.277, 'rigs': 0.277, 'supplies': 0.229, 'involved': 0.223, 'oil': 0.205, 'gas': 0.192}
integrated oil gas
{'integrated': 0.653, 'oil': 0.553, 'gas': 0.518}
integrated oil companies engaged in the exploration production of oil and gas as well as at least one other significant activity in either refining marketing and transportation or chemicals 
{'oil': 0.403, 'one': 0.271, 'least': 0.271, 'well': 0.25

financial institutions primarily engaged in investment management and or related custody and securities fee based services includes companies operating mutual funds closed end funds and unit investment trusts excludes banks and other financial institutions primarily involved in commercial lending investment banking brokerage and other specialized financial activities 
{'investment': 0.419, 'financial': 0.372, 'funds': 0.351, 'institutions': 0.288, 'primarily': 0.18, 'unit': 0.175, 'mutual': 0.175, 'fee': 0.175, 'closed': 0.175, 'end': 0.156}
investment banking brokerage
{'investment': 0.588, 'brokerage': 0.572, 'banking': 0.572}
financial institutions primarily engaged in investment banking brokerage services including equity and debt underwriting mergers and acquisitions securities lending and advisory services excludes banks and other financial institutions primarily involved in commercial lending asset management and specialized financial activities 
{'financial': 0.426, 'institutio

In [20]:
dict_new

{'drilling': 0.525,
 'wells': 0.277,
 'rigs': 0.277,
 'contractors': 0.307,
 'contract': 0.222,
 'owners': 0.124,
 'services': 0.332,
 'oil': 0.341,
 'gas': 0.144,
 'equipment': 0.404,
 'evaluation': 0.296,
 'completion': 0.296,
 'supplies': 0.43,
 'involved': 0.414,
 'integrated': 0.641,
 'one': 0.194,
 'least': 0.271,
 'well': 0.207,
 'refining': 0.265,
 'either': 0.258,
 'activity': 0.258,
 'significant': 0.247,
 'exploration': 0.279,
 'production': 0.227,
 'elsewhere': 0.379,
 'engaged': 0.25,
 'classified': 0.245,
 'companies': 0.223,
 'marketing': 0.369,
 'traders': 0.511,
 'refined': 0.36,
 'independent': 0.485,
 'energy': 0.234,
 'power': 0.221,
 'transportation': 0.236,
 'storage': 0.232,
 'pipelines': 0.407,
 'slurry': 0.203,
 'shipping': 0.203,
 'product': 0.203,
 'midstream': 0.203,
 'natural': 0.19,
 'fuels': 0.592,
 'consumable': 0.592,
 'coal': 0.272,
 'gases': 0.696,
 'mining': 0.256,
 'primarily': 0.287,
 'related': 0.27,
 'metallurgical': 0.31,
 'generation': 0.192,
 

In [21]:
dict_new.keys()

dict_keys(['drilling', 'wells', 'rigs', 'contractors', 'contract', 'owners', 'services', 'oil', 'gas', 'equipment', 'evaluation', 'completion', 'supplies', 'involved', 'integrated', 'one', 'least', 'well', 'refining', 'either', 'activity', 'significant', 'exploration', 'production', 'elsewhere', 'engaged', 'classified', 'companies', 'marketing', 'traders', 'refined', 'independent', 'energy', 'power', 'transportation', 'storage', 'pipelines', 'slurry', 'shipping', 'product', 'midstream', 'natural', 'fuels', 'consumable', 'coal', 'gases', 'mining', 'primarily', 'related', 'metallurgical', 'generation', 'coking', 'commodity', 'chemicals', 'industrial', 'synthetic', 'plastics', 'petrochemicals', 'films', 'fibers', 'explosives', 'basic', 'pigments', 'diversified', 'chemical', 'range', 'fertilizers', 'specialty', 'agricultural', 'industries', 'potash', 'pesticides', 'agriculture', 'producers', 'manufacturers', 'variety', 'value', 'sealants', 'polymers', 'fine', 'coatings', 'advanced', 'adhes

In [22]:
#import json

#with open("keywordsnew.txt", "w") as f:
#    json.dump(dict, f)

In [23]:
dict_new.items()

dict_items([('drilling', 0.525), ('wells', 0.277), ('rigs', 0.277), ('contractors', 0.307), ('contract', 0.222), ('owners', 0.124), ('services', 0.332), ('oil', 0.341), ('gas', 0.144), ('equipment', 0.404), ('evaluation', 0.296), ('completion', 0.296), ('supplies', 0.43), ('involved', 0.414), ('integrated', 0.641), ('one', 0.194), ('least', 0.271), ('well', 0.207), ('refining', 0.265), ('either', 0.258), ('activity', 0.258), ('significant', 0.247), ('exploration', 0.279), ('production', 0.227), ('elsewhere', 0.379), ('engaged', 0.25), ('classified', 0.245), ('companies', 0.223), ('marketing', 0.369), ('traders', 0.511), ('refined', 0.36), ('independent', 0.485), ('energy', 0.234), ('power', 0.221), ('transportation', 0.236), ('storage', 0.232), ('pipelines', 0.407), ('slurry', 0.203), ('shipping', 0.203), ('product', 0.203), ('midstream', 0.203), ('natural', 0.19), ('fuels', 0.592), ('consumable', 0.592), ('coal', 0.272), ('gases', 0.696), ('mining', 0.256), ('primarily', 0.287), ('rel

In [24]:
temp = []
dictList = []

#My attempt:
for key, value in dict_new.items():
    aKey = key
    aValue = value
    temp.append(aKey)
    temp.append(aValue)
    dictList.append(temp) 
    aKey = ""
    aValue = ""

In [25]:
import re
import string
frequency = {}
dictword = dict_new.keys()



In [26]:
import os
from os import listdir
from os.path import isfile, join

newspath = 'Weekly_news'

newsfiles = [f for f in listdir(newspath) if isfile(join(newspath, f))]

for file in newsfiles:
    filepath = newspath + "/" + file
    data = open(filepath,'r',encoding='utf-8')
    text = data.read()
    text_string = data.read().lower()




for raw_str in dictword:
    pat = re.compile(raw_str)
    for w in text_string.split():
        if pat.match(w):
            print ("Keywords:", raw_str, "with TextFileWords:", w)
            count = frequency.get(w,0)
            frequency[w] = count + 1
            frequency_list = frequency.keys()
 
            for w in frequency_list:
                print(w, frequency[w])

In [30]:
document_text = open('Weekly_news/00.txt', 'r')
text_string = document_text.read().lower()

for raw_str in dictword:
    pat = re.compile(raw_str)
    for w in text_string.split():
        if pat.match(w):
            print ("Keywords:", raw_str, "-----","with TextFileWords:", w)
            count = frequency.get(raw_str,0)
            frequency[raw_str] = count + 1
            frequency_list = frequency.keys()
 
            for w in frequency_list:
                 print(w, frequency[w])

Keywords: contract ----- with TextFileWords: contract
contract 3
oil 2
gas 6
one 18
least 10
power 14
coal 6
coalition 4
chemical 6
rangers 2
mine 2
semiconductor 16
subway 2
rockets 4
south 22
southgate 4
africans 2
african 4
invest 12
investors 2
defense 2
civilians 2
large 2
heavy 6
stake 4
machinery 16
water 4
pollution 4
officer 2
office 8
security 14
protection 12
employment 4
human 6
airline 2
air 2
airliner 2
airport 4
passenger 2
landmark 8
vehicle 4
roadside 2
auto 8
lightning 2
motorcycle 4
three 4
home 4
game 6
wall 2
hand 2
video 2
tourist 12
closely 4
close 8
closed 4
music 2
bookshop 8
tv 6
ordered 2
computer 4
supply 4
market 6
caretaker 4
carey 6
healthily 2
eye 2
sector 4
nationalism 6
national 6
smaller 8
medium 6
financial 6
revenue 4
central 2
credit 2
money 4
united 8
unity 4
endorses 4
end 14
brokerage 2
lending 8
capitals 4
capital 12
purchase 2
service 12
back 10
infrastructure 2
test 6
signal 4
highway 12
higher 4
highlights 2
network 4
telecommunication 4
gam

lending 8
capitals 4
capital 12
purchase 2
service 12
back 10
infrastructure 2
test 6
signal 4
highway 12
higher 4
highlights 2
network 4
telecommunication 4
gambling 4
search 6
peru 6
percent 8
persist 4
pay 2
payment 6
utility 8
two 12
across 4
Keywords: one ----- with TextFileWords: one
contract 3
oil 3
gas 9
one 23
least 10
power 14
coal 6
coalition 4
chemical 6
rangers 2
mine 2
semiconductor 16
subway 2
rockets 4
south 22
southgate 4
africans 2
african 4
invest 12
investors 2
defense 2
civilians 2
large 2
heavy 6
stake 4
machinery 16
water 4
pollution 4
officer 2
office 8
security 14
protection 12
employment 4
human 6
airline 2
air 2
airliner 2
airport 4
passenger 2
landmark 8
vehicle 4
roadside 2
auto 8
lightning 2
motorcycle 4
three 4
home 4
game 6
wall 2
hand 2
video 2
tourist 12
closely 4
close 8
closed 4
music 2
bookshop 8
tv 6
ordered 2
computer 4
supply 4
market 6
caretaker 4
carey 6
healthily 2
eye 2
sector 4
nationalism 6
national 6
smaller 8
medium 6
financial 6
revenue 

coal 6
coalition 4
chemical 6
rangers 2
mine 2
semiconductor 16
subway 2
rockets 4
south 22
southgate 4
africans 2
african 4
invest 12
investors 2
defense 2
civilians 2
large 2
heavy 6
stake 4
machinery 16
water 4
pollution 4
officer 2
office 8
security 14
protection 12
employment 4
human 6
airline 2
air 2
airliner 2
airport 4
passenger 2
landmark 8
vehicle 4
roadside 2
auto 8
lightning 2
motorcycle 4
three 4
home 4
game 6
wall 2
hand 2
video 2
tourist 12
closely 4
close 8
closed 4
music 2
bookshop 8
tv 6
ordered 2
computer 4
supply 4
market 6
caretaker 4
carey 6
healthily 2
eye 2
sector 4
nationalism 6
national 6
smaller 8
medium 6
financial 6
revenue 4
central 2
credit 2
money 4
united 8
unity 4
endorses 4
end 14
brokerage 2
lending 8
capitals 4
capital 12
purchase 2
service 12
back 10
infrastructure 2
test 6
signal 4
highway 12
higher 4
highlights 2
network 4
telecommunication 4
gambling 4
search 6
peru 6
percent 8
persist 4
pay 2
payment 6
utility 8
two 12
across 4
Keywords: power 

passenger 2
landmark 8
vehicle 4
roadside 2
auto 8
lightning 2
motorcycle 4
three 4
home 4
game 6
wall 2
hand 2
video 2
tourist 12
closely 4
close 8
closed 4
music 2
bookshop 8
tv 6
ordered 2
computer 4
supply 4
market 6
caretaker 4
carey 6
healthily 2
eye 2
sector 4
nationalism 6
national 6
smaller 8
medium 6
financial 6
revenue 4
central 2
credit 2
money 4
united 8
unity 4
endorses 4
end 14
brokerage 2
lending 8
capitals 4
capital 12
purchase 2
service 12
back 10
infrastructure 2
test 6
signal 4
highway 12
higher 4
highlights 2
network 4
telecommunication 4
gambling 4
search 6
peru 6
percent 8
persist 4
pay 2
payment 6
utility 8
two 12
across 4
Keywords: coal ----- with TextFileWords: coal
contract 3
oil 3
gas 9
one 27
least 15
power 21
coal 9
coalition 4
chemical 6
rangers 2
mine 2
semiconductor 16
subway 2
rockets 4
south 22
southgate 4
africans 2
african 4
invest 12
investors 2
defense 2
civilians 2
large 2
heavy 6
stake 4
machinery 16
water 4
pollution 4
officer 2
office 8
securi

Keywords: mine ----- with TextFileWords: mine
contract 3
oil 3
gas 9
one 27
least 15
power 21
coal 11
coalition 4
chemical 9
rangers 2
mine 3
semiconductor 16
subway 2
rockets 4
south 22
southgate 4
africans 2
african 4
invest 12
investors 2
defense 2
civilians 2
large 2
heavy 6
stake 4
machinery 16
water 4
pollution 4
officer 2
office 8
security 14
protection 12
employment 4
human 6
airline 2
air 2
airliner 2
airport 4
passenger 2
landmark 8
vehicle 4
roadside 2
auto 8
lightning 2
motorcycle 4
three 4
home 4
game 6
wall 2
hand 2
video 2
tourist 12
closely 4
close 8
closed 4
music 2
bookshop 8
tv 6
ordered 2
computer 4
supply 4
market 6
caretaker 4
carey 6
healthily 2
eye 2
sector 4
nationalism 6
national 6
smaller 8
medium 6
financial 6
revenue 4
central 2
credit 2
money 4
united 8
unity 4
endorses 4
end 14
brokerage 2
lending 8
capitals 4
capital 12
purchase 2
service 12
back 10
infrastructure 2
test 6
signal 4
highway 12
higher 4
highlights 2
network 4
telecommunication 4
gambling 4

...

financial 9
revenue 6
central 3
credit 3
money 6
united 8
unity 4
endorses 4
end 23
brokerage 3
lending 12
capitals 4
capital 20
purchase 3
service 18
back 15
infrastructure 3
test 9
signal 6
highway 12
higher 4
highlights 2
network 4
telecommunication 4
gambling 4
search 6
peru 6
percent 8
persist 4
pay 2
payment 6
utility 8
two 12
across 4
range 1
semi 4
sub 1
rock 2
civil 1
machine 4
land 4
roads 1
light 1
tour 6
books 2
order 1
book 2
care 5
health 1
small 4
unit 6
high 3
Keywords: high ----- with TextFileWords: highway
contract 3
oil 3
gas 9
one 27
least 15
power 21
coal 11
coalition 4
chemical 9
rangers 2
mine 3
semiconductor 20
subway 2
rockets 4
south 35
southgate 4
africans 2
african 7
invest 19
investors 2
defense 3
civilians 2
large 3
heavy 9
stake 6
machinery 20
water 6
pollution 6
officer 2
office 13
security 21
protection 18
employment 6
human 9
airline 2
air 6
airliner 2
airport 5
passenger 3
landmark 8
vehicle 6
roadside 2
auto 12
lightning 2
motorcycle 6
three 6
home 6

home 6
game 9
wall 3
hand 3
video 3
tourist 12
closely 4
close 15
closed 5
music 3
bookshop 8
tv 9
ordered 2
computer 6
supply 6
market 9
caretaker 4
carey 6
healthily 2
eye 3
sector 6
nationalism 6
national 12
smaller 8
medium 9
financial 9
revenue 6
central 3
credit 3
money 6
united 8
unity 4
endorses 4
end 23
brokerage 3
lending 12
capitals 4
capital 20
purchase 3
service 18
back 15
infrastructure 3
test 9
signal 6
highway 12
higher 4
highlights 2
network 6
telecommunication 6
gambling 4
search 6
peru 6
percent 8
persist 4
pay 2
payment 6
utility 8
two 12
across 4
range 1
semi 4
sub 1
rock 2
civil 1
machine 4
land 4
roads 1
light 1
tour 6
books 2
order 1
book 2
care 5
health 1
small 4
unit 6
high 9
Keywords: gambling ----- with TextFileWords: gambling
contract 3
oil 3
gas 9
one 27
least 15
power 21
coal 11
coalition 4
chemical 9
rangers 2
mine 3
semiconductor 20
subway 2
rockets 4
south 35
southgate 4
africans 2
african 7
invest 19
investors 2
defense 3
civilians 2
large 3
heavy 9
s

landmark 8
vehicle 6
roadside 2
auto 12
lightning 2
motorcycle 6
three 6
home 6
game 9
wall 3
hand 3
video 3
tourist 12
closely 4
close 15
closed 5
music 3
bookshop 8
tv 9
ordered 2
computer 6
supply 6
market 9
caretaker 4
carey 6
healthily 2
eye 3
sector 6
nationalism 6
national 12
smaller 8
medium 9
financial 9
revenue 6
central 3
credit 3
money 6
united 8
unity 4
endorses 4
end 23
brokerage 3
lending 12
capitals 4
capital 20
purchase 3
service 18
back 15
infrastructure 3
test 9
signal 6
highway 12
higher 4
highlights 2
network 6
telecommunication 6
gambling 6
search 9
peru 6
percent 8
persist 4
pay 2
payment 6
utility 8
two 12
across 4
range 1
semi 4
sub 1
rock 2
civil 1
machine 4
land 4
roads 1
light 1
tour 6
books 2
order 1
book 2
care 5
health 1
small 4
unit 6
high 9
per 3
Keywords: per ----- with TextFileWords: percent
contract 3
oil 3
gas 9
one 27
least 15
power 21
coal 11
coalition 4
chemical 9
rangers 2
mine 3
semiconductor 20
subway 2
rockets 4
south 35
southgate 4
africans 

pollution 6
officer 2
office 13
security 21
protection 18
employment 6
human 9
airline 2
air 6
airliner 2
airport 5
passenger 3
landmark 8
vehicle 6
roadside 2
auto 12
lightning 2
motorcycle 6
three 6
home 6
game 9
wall 3
hand 3
video 3
tourist 12
closely 4
close 15
closed 5
music 3
bookshop 8
tv 9
ordered 2
computer 6
supply 6
market 9
caretaker 4
carey 6
healthily 2
eye 3
sector 6
nationalism 6
national 12
smaller 8
medium 9
financial 9
revenue 6
central 3
credit 3
money 6
united 8
unity 4
endorses 4
end 23
brokerage 3
lending 12
capitals 4
capital 20
purchase 3
service 18
back 15
infrastructure 3
test 9
signal 6
highway 12
higher 4
highlights 2
network 6
telecommunication 6
gambling 6
search 9
peru 6
percent 8
persist 4
pay 4
payment 6
utility 8
two 12
across 4
range 1
semi 4
sub 1
rock 2
civil 1
machine 4
land 4
roads 1
light 1
tour 6
books 2
order 1
book 2
care 5
health 1
small 4
unit 6
high 9
per 9
Keywords: pay ----- with TextFileWords: payment
contract 3
oil 3
gas 9
one 27
leas

southgate 4
africans 2
african 7
invest 19
investors 2
defense 3
civilians 2
large 3
heavy 9
stake 6
machinery 20
water 6
pollution 6
officer 2
office 13
security 21
protection 18
employment 6
human 9
airline 2
air 6
airliner 2
airport 5
passenger 3
landmark 8
vehicle 6
roadside 2
auto 12
lightning 2
motorcycle 6
three 6
home 6
game 9
wall 3
hand 3
video 3
tourist 12
closely 4
close 15
closed 5
music 3
bookshop 8
tv 9
ordered 2
computer 6
supply 6
market 9
caretaker 4
carey 6
healthily 2
eye 3
sector 6
nationalism 6
national 12
smaller 8
medium 9
financial 9
revenue 6
central 3
credit 3
money 6
united 8
unity 4
endorses 4
end 23
brokerage 3
lending 12
capitals 4
capital 20
purchase 3
service 18
back 15
infrastructure 3
test 9
signal 6
highway 12
higher 4
highlights 2
network 6
telecommunication 6
gambling 6
search 9
peru 6
percent 8
persist 4
pay 6
payment 6
utility 12
two 14
across 4
range 1
semi 4
sub 1
rock 2
civil 1
machine 4
land 4
roads 1
light 1
tour 6
books 2
order 1
book 2
car

In [31]:
document_text = open('Weekly_news/00.txt', 'r')
text_string = document_text.read().lower()

for raw_str in dictword:
    pat = re.compile(raw_str)
    for w in text_string.split():
        if pat.match(w):
            print ("Keywords:", raw_str, "-----","with TextFileWords:", w)
            count = frequency.get(w,0)
            frequency[w] = count + 1
            frequency_list = frequency.keys()
 
            for w in frequency_list:
                 print(w, frequency[w])

Keywords: contract ----- with TextFileWords: contract
contract 4
oil 3
gas 9
one 27
least 15
power 21
coal 11
coalition 4
chemical 9
rangers 2
mine 3
semiconductor 20
subway 2
rockets 4
south 35
southgate 4
africans 2
african 7
invest 19
investors 2
defense 3
civilians 2
large 3
heavy 9
stake 6
machinery 20
water 6
pollution 6
officer 2
office 13
security 21
protection 18
employment 6
human 9
airline 2
air 6
airliner 2
airport 5
passenger 3
landmark 8
vehicle 6
roadside 2
auto 12
lightning 2
motorcycle 6
three 6
home 6
game 9
wall 3
hand 3
video 3
tourist 12
closely 4
close 15
closed 5
music 3
bookshop 8
tv 9
ordered 2
computer 6
supply 6
market 9
caretaker 4
carey 6
healthily 2
eye 3
sector 6
nationalism 6
national 12
smaller 8
medium 9
financial 9
revenue 6
central 3
credit 3
money 6
united 8
unity 4
endorses 4
end 23
brokerage 3
lending 12
capitals 4
capital 20
purchase 3
service 18
back 15
infrastructure 3
test 9
signal 6
highway 12
higher 4
highlights 2
network 6
telecommunication

tour 6
books 2
order 1
book 2
care 5
health 1
small 4
unit 6
high 9
per 9
Keywords: one ----- with TextFileWords: one
contract 4
oil 4
gas 12
one 32
least 15
power 21
coal 11
coalition 4
chemical 9
rangers 2
mine 3
semiconductor 20
subway 2
rockets 4
south 35
southgate 4
africans 2
african 7
invest 19
investors 2
defense 3
civilians 2
large 3
heavy 9
stake 6
machinery 20
water 6
pollution 6
officer 2
office 13
security 21
protection 18
employment 6
human 9
airline 2
air 6
airliner 2
airport 5
passenger 3
landmark 8
vehicle 6
roadside 2
auto 12
lightning 2
motorcycle 6
three 6
home 6
game 9
wall 3
hand 3
video 3
tourist 12
closely 4
close 15
closed 5
music 3
bookshop 8
tv 9
ordered 2
computer 6
supply 6
market 9
caretaker 4
carey 6
healthily 2
eye 3
sector 6
nationalism 6
national 12
smaller 8
medium 9
financial 9
revenue 6
central 3
credit 3
money 6
united 8
unity 4
endorses 4
end 23
brokerage 3
lending 12
capitals 4
capital 20
purchase 3
service 18
back 15
infrastructure 3
test 9
sign

utility 12
two 18
across 6
range 1
semi 4
sub 1
rock 2
civil 1
machine 4
land 4
roads 1
light 1
tour 6
books 2
order 1
book 2
care 5
health 1
small 4
unit 6
high 9
per 9
Keywords: least ----- with TextFileWords: least
contract 4
oil 4
gas 12
one 36
least 19
power 21
coal 11
coalition 4
chemical 9
rangers 2
mine 3
semiconductor 20
subway 2
rockets 4
south 35
southgate 4
africans 2
african 7
invest 19
investors 2
defense 3
civilians 2
large 3
heavy 9
stake 6
machinery 20
water 6
pollution 6
officer 2
office 13
security 21
protection 18
employment 6
human 9
airline 2
air 6
airliner 2
airport 5
passenger 3
landmark 8
vehicle 6
roadside 2
auto 12
lightning 2
motorcycle 6
three 6
home 6
game 9
wall 3
hand 3
video 3
tourist 12
closely 4
close 15
closed 5
music 3
bookshop 8
tv 9
ordered 2
computer 6
supply 6
market 9
caretaker 4
carey 6
healthily 2
eye 3
sector 6
nationalism 6
national 12
smaller 8
medium 9
financial 9
revenue 6
central 3
credit 3
money 6
united 8
unity 4
endorses 4
end 23
bro

highway 12
higher 4
highlights 2
network 6
telecommunication 6
gambling 6
search 9
peru 6
percent 8
persist 4
pay 6
payment 6
utility 12
two 18
across 6
range 1
semi 4
sub 1
rock 2
civil 1
machine 4
land 4
roads 1
light 1
tour 6
books 2
order 1
book 2
care 5
health 1
small 4
unit 6
high 9
per 9
Keywords: power ----- with TextFileWords: power
contract 4
oil 4
gas 12
one 36
least 20
power 28
coal 11
coalition 4
chemical 9
rangers 2
mine 3
semiconductor 20
subway 2
rockets 4
south 35
southgate 4
africans 2
african 7
invest 19
investors 2
defense 3
civilians 2
large 3
heavy 9
stake 6
machinery 20
water 6
pollution 6
officer 2
office 13
security 21
protection 18
employment 6
human 9
airline 2
air 6
airliner 2
airport 5
passenger 3
landmark 8
vehicle 6
roadside 2
auto 12
lightning 2
motorcycle 6
three 6
home 6
game 9
wall 3
hand 3
video 3
tourist 12
closely 4
close 15
closed 5
music 3
bookshop 8
tv 9
ordered 2
computer 6
supply 6
market 9
caretaker 4
carey 6
healthily 2
eye 3
sector 6
nation

africans 2
african 7
invest 19
investors 2
defense 3
civilians 2
large 3
heavy 9
stake 6
machinery 20
water 6
pollution 6
officer 2
office 13
security 21
protection 18
employment 6
human 9
airline 2
air 6
airliner 2
airport 5
passenger 3
landmark 8
vehicle 6
roadside 2
auto 12
lightning 2
motorcycle 6
three 6
home 6
game 9
wall 3
hand 3
video 3
tourist 12
closely 4
close 15
closed 5
music 3
bookshop 8
tv 9
ordered 2
computer 6
supply 6
market 9
caretaker 4
carey 6
healthily 2
eye 3
sector 6
nationalism 6
national 12
smaller 8
medium 9
financial 9
revenue 6
central 3
credit 3
money 6
united 8
unity 4
endorses 4
end 23
brokerage 3
lending 12
capitals 4
capital 20
purchase 3
service 18
back 15
infrastructure 3
test 9
signal 6
highway 12
higher 4
highlights 2
network 6
telecommunication 6
gambling 6
search 9
peru 6
percent 8
persist 4
pay 6
payment 6
utility 12
two 18
across 6
range 1
semi 4
sub 1
rock 2
civil 1
machine 4
land 4
roads 1
light 1
tour 6
books 2
order 1
book 2
care 5
health 1

...

tour 6
books 2
order 1
book 2
care 5
health 1
small 4
unit 6
high 9
per 9
Keywords: high ----- with TextFileWords: higher
contract 4
oil 4
gas 12
one 36
least 20
power 28
coal 14
coalition 6
chemical 12
rangers 3
mine 4
semiconductor 28
subway 3
rockets 6
south 46
southgate 6
africans 3
african 9
invest 25
investors 3
defense 4
civilians 3
large 4
heavy 12
stake 8
machinery 28
water 8
pollution 8
officer 3
office 17
security 28
protection 24
employment 8
human 12
airline 3
air 7
airliner 3
airport 7
passenger 4
landmark 12
vehicle 8
roadside 3
auto 16
lightning 3
motorcycle 8
three 8
home 8
game 12
wall 4
hand 4
video 4
tourist 18
closely 6
close 19
closed 7
music 4
bookshop 12
tv 12
ordered 3
computer 8
supply 8
market 12
caretaker 6
carey 9
healthily 3
eye 4
sector 8
nationalism 9
national 15
smaller 12
medium 12
financial 12
revenue 8
central 4
credit 4
money 8
united 12
unity 6
endorses 6
end 30
brokerage 4
lending 16
capitals 6
capital 26
purchase 4
service 24
back 20
infrastructu

payment 6
utility 12
two 18
across 6
range 1
semi 4
sub 1
rock 2
civil 1
machine 4
land 4
roads 1
light 1
tour 6
books 2
order 1
book 2
care 5
health 1
small 4
unit 6
high 9
per 9
Keywords: telecommunication ----- with TextFileWords: telecommunication
contract 4
oil 4
gas 12
one 36
least 20
power 28
coal 14
coalition 6
chemical 12
rangers 3
mine 4
semiconductor 28
subway 3
rockets 6
south 46
southgate 6
africans 3
african 9
invest 25
investors 3
defense 4
civilians 3
large 4
heavy 12
stake 8
machinery 28
water 8
pollution 8
officer 3
office 17
security 28
protection 24
employment 8
human 12
airline 3
air 7
airliner 3
airport 7
passenger 4
landmark 12
vehicle 8
roadside 3
auto 16
lightning 3
motorcycle 8
three 8
home 8
game 12
wall 4
hand 4
video 4
tourist 18
closely 6
close 19
closed 7
music 4
bookshop 12
tv 12
ordered 3
computer 8
supply 8
market 12
caretaker 6
carey 9
healthily 3
eye 4
sector 8
nationalism 9
national 15
smaller 12
medium 12
financial 12
revenue 8
central 4
credit 4
m

semi 4
sub 1
rock 2
civil 1
machine 4
land 4
roads 1
light 1
tour 6
books 2
order 1
book 2
care 5
health 1
small 4
unit 6
high 9
per 9
Keywords: per ----- with TextFileWords: peru
contract 4
oil 4
gas 12
one 36
least 20
power 28
coal 14
coalition 6
chemical 12
rangers 3
mine 4
semiconductor 28
subway 3
rockets 6
south 46
southgate 6
africans 3
african 9
invest 25
investors 3
defense 4
civilians 3
large 4
heavy 12
stake 8
machinery 28
water 8
pollution 8
officer 3
office 17
security 28
protection 24
employment 8
human 12
airline 3
air 7
airliner 3
airport 7
passenger 4
landmark 12
vehicle 8
roadside 3
auto 16
lightning 3
motorcycle 8
three 8
home 8
game 12
wall 4
hand 4
video 4
tourist 18
closely 6
close 19
closed 7
music 4
bookshop 12
tv 12
ordered 3
computer 8
supply 8
market 12
caretaker 6
carey 9
healthily 3
eye 4
sector 8
nationalism 9
national 15
smaller 12
medium 12
financial 12
revenue 8
central 4
credit 4
money 8
united 12
unity 6
endorses 6
end 30
brokerage 4
lending 16
capita

credit 4
money 8
united 12
unity 6
endorses 6
end 30
brokerage 4
lending 16
capitals 6
capital 26
purchase 4
service 24
back 20
infrastructure 4
test 12
signal 8
highway 18
higher 6
highlights 3
network 8
telecommunication 8
gambling 8
search 12
peru 9
percent 12
persist 6
pay 7
payment 8
utility 12
two 18
across 6
range 1
semi 4
sub 1
rock 2
civil 1
machine 4
land 4
roads 1
light 1
tour 6
books 2
order 1
book 2
care 5
health 1
small 4
unit 6
high 9
per 9
Keywords: pay ----- with TextFileWords: payment
contract 4
oil 4
gas 12
one 36
least 20
power 28
coal 14
coalition 6
chemical 12
rangers 3
mine 4
semiconductor 28
subway 3
rockets 6
south 46
southgate 6
africans 3
african 9
invest 25
investors 3
defense 4
civilians 3
large 4
heavy 12
stake 8
machinery 28
water 8
pollution 8
officer 3
office 17
security 28
protection 24
employment 8
human 12
airline 3
air 7
airliner 3
airport 7
passenger 4
landmark 12
vehicle 8
roadside 3
auto 16
lightning 3
motorcycle 8
three 8
home 8
game 12
wall 4
h

caretaker 6
carey 9
healthily 3
eye 4
sector 8
nationalism 9
national 15
smaller 12
medium 12
financial 12
revenue 8
central 4
credit 4
money 8
united 12
unity 6
endorses 6
end 30
brokerage 4
lending 16
capitals 6
capital 26
purchase 4
service 24
back 20
infrastructure 4
test 12
signal 8
highway 18
higher 6
highlights 3
network 8
telecommunication 8
gambling 8
search 12
peru 9
percent 12
persist 6
pay 7
payment 9
utility 16
two 21
across 6
range 1
semi 4
sub 1
rock 2
civil 1
machine 4
land 4
roads 1
light 1
tour 6
books 2
order 1
book 2
care 5
health 1
small 4
unit 6
high 9
per 9
Keywords: two ----- with TextFileWords: two
contract 4
oil 4
gas 12
one 36
least 20
power 28
coal 14
coalition 6
chemical 12
rangers 3
mine 4
semiconductor 28
subway 3
rockets 6
south 46
southgate 6
africans 3
african 9
invest 25
investors 3
defense 4
civilians 3
large 4
heavy 12
stake 8
machinery 28
water 8
pollution 8
officer 3
office 17
security 28
protection 24
employment 8
human 12
airline 3
air 7
airline